In [3]:
import openai 
from utils import call_chatgpt_api_all_chats, stream_process_chatgpt_response
from secret import gao_key as key 
from resources.generate_response import analyze_situation_rag, analyze_situation_rag_guidance
import concurrent.futures
import re
import time 
import json 
from benefits.eligibility_check import eligibility_check

openai.api_key = key

# Orchestration 1

## Step 1: High-Level Reasoning (Meta-Reasoning Step)

### Introduce a preliminary reasoning step, asking the LLM to reason broadly about the situation and explicitly articulate what's required or relevant for each subsequent module


In [25]:
# Define meta-reasoning prompt 
def generate_meta_reasoning_prompt(situation: str) -> str:
    return f"""
You are an expert mental health support agent tasked with analyzing a user's provided situation to guide subsequent analysis modules. Your goal is NOT to produce concrete SMART goals, resources, or specific questions directly, but rather to reason conceptually and give clear directions on how each specialized module should approach the situation.

Given the user's situation below, clearly articulate conceptual guidance and direction ONLY for each of the following modules:

1. **SMART Goals Module**: 
   - What key themes or areas should SMART goals focus on, given this situation?
   - Briefly outline what characteristics or considerations the SMART goals should prioritize, without listing actual goals.

2. **Resources Module**:
   - Identify types or categories of resources that are conceptually relevant or could potentially benefit the user.
   - Specify the criteria or conditions to use when searching for relevant resources.

3. **Follow-up Questions Module**:
   - Conceptually suggest what kinds of insightful follow-up questions would be helpful.
   - Highlight themes or areas that need clarification or further exploration.

4. **Benefits Eligibility Module**:
   - Provide high-level guidance on what types of benefits or eligibility considerations should be examined.
   - Mention what contextual clues or criteria would be helpful to determine eligibility.

### User Situation:
{situation}

### Provide structured, conceptual reasoning clearly labeled for each module:
"""

In [64]:
# o3-mini for enhanced reasoning capability
# lower price than 4o but more expensive than 4o-mini ($15 / 1M tokens)
# o3-mini is only used during the reasoning step

def call_llm_reason(messages, model="o3-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

# general purpose LLM
def call_llm(messages, model="gpt-4o-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

def get_meta_reasoning(situation: str):
    meta_prompt = generate_meta_reasoning_prompt(situation)

    messages = [
        {"role": "system", "content": "You are an expert mental health reasoning assistant providing structured conceptual guidance for mental health support modules."},
        {"role": "user", "content": meta_prompt}
    ]

    reasoning_response = call_llm_reason(messages)
    structured_reasoning = parse_reasoning(reasoning_response)
    
    return structured_reasoning

In [34]:
# Parsing function to neatly structure the response
def parse_reasoning(response_text: str):
    modules = ["SMART Goals Module", "Resources Module", "Follow-up Questions Module", "Benefits Eligibility Module"]
    reasoning = {}

    current_module = None
    for line in response_text.splitlines():
        line_stripped = line.strip()
        if any(module in line_stripped for module in modules):
            current_module = next(module for module in modules if module in line_stripped)
            reasoning[current_module] = ""
        elif current_module and line_stripped != "":
            reasoning[current_module] += line_stripped + " "

    # Clean extra spaces
    for key in reasoning:
        reasoning[key] = reasoning[key].strip()

    return reasoning

In [35]:
example_situation = "A 72-year-old retired man in Morristown, NJ, \
living in permanent housing and dually enrolled (Medicaid/Medicare), \
seeks support for maintaining his physical wellness through better diet and exercise. \
He is a non-drug user with ID and no incarceration history but struggles with isolation, \
spending most of his days alone in his room, and wants to build social connections."

In [38]:
meta_reasoning_result = get_meta_reasoning(example_situation)
print("\n=== Structured Meta-Reasoning Output ===\n")
for module, guidance in meta_reasoning_result.items():
    print(f"**{module}**:\n{guidance}\n")


=== Structured Meta-Reasoning Output ===

**SMART Goals Module**:
• Conceptual Themes: – Physical health improvements: Create focus on how diet and exercise can be tailored to his age-related needs. – Social connection development: Address the goal of breaking isolation by integrating social activities. – Holistic wellness: Consider mental and emotional well-being given the impact of isolation on overall health. • Considerations for Characteristics: – Specificity relative to measurable physical activities and nutritional changes suitable for a 72-year-old. – Attainable milestones given his current lifestyle and potential hesitations due to isolation and change. – Realistic steps that interweave physical and social goals in practical, locally available activities. – Temporally defined checkpoints that align with his pace of adaptation and sustainable change. – Relevance to his dual health enrollment, ensuring any health-related goals are in line with his benefits and local services. ──

In [41]:
meta_reasoning_result.keys()

dict_keys(['SMART Goals Module', 'Resources Module', 'Follow-up Questions Module', 'Benefits Eligibility Module'])

# Step 2: Module-specific LLM Calls with Reasoning Context

##### Feed the initial meta-reasoning output, original module prompts and the user situation to each module 

## SMART Goals

In [42]:
smart_goal_reasoning = meta_reasoning_result['SMART Goals Module']
resource_reasoning = meta_reasoning_result['Resources Module']
question_reasoning = meta_reasoning_result['Follow-up Questions Module']
benefit_reasonsing = meta_reasoning_result['Benefits Eligibility Module']

In [43]:
mental_health_system_prompt = open("mental_health/prompts/mental_health_prompt.txt").read()
question_prompt = open("mental_health/prompts/question_prompts.txt").read()
benefit_prompt = open("benefits/prompts/uncertain_prompt_wellness.txt").read()
resource_prompt = open("mental_health/prompts/resource_prompt.txt").read()

In [48]:
def format_smart_goals(raw_text):
    formatted_text = raw_text.replace("\u2002", " ").replace("\\n", "\n")
    formatted_text = re.sub(r'_{2,}', '\n' + '-'*50 + '\n', formatted_text)
    formatted_text = re.sub(r'(Specific:|Measurable:|Achievable:|Realistic:|Timely:)', r'\n  \1', formatted_text)
    formatted_text = re.sub(r'\n+', '\n', formatted_text).strip()
    return formatted_text
    
def get_smart_goals(reasoning_guidance, situation):
    messages = [
        {'role': 'system', 'content': mental_health_system_prompt},
        {'role': 'assistant', 'content': reasoning_guidance},
        {'role': 'user', 'content': situation}
    ]
    
    return call_llm(messages)

In [49]:
results = get_smart_goals(smart_goal_reasoning, example_situation)
results = format_smart_goals(results)
print(results)

Below are some SMART (Specific, Measurable, Achievable, Realistic, and Timely) goals that address both your desire to maintain physical wellness through a better diet and exercise routine, along with building social connections to reduce isolation.
──────────────────────────────
1. Physical Exercise Goal
• 
  Specific: Aim to get 30 minutes of moderate physical activity daily, such as walking around your neighborhood or using a community park in Morristown, NJ.  
• 
  Measurable: Track your walks by noting start and end times or using a simple step counter app.  
• 
  Achievable: Start with a pace that's comfortable for you, gradually increasing your activity as your stamina improves.  
• 
  Realistic: Given your current living situation and being retired, a daily walk is a manageable activity that won’t require a major lifestyle change.  
• 
  Timely: Begin immediately and commit to this routine for the next 4 weeks as a starting point. Reassess at the end of the month to consider inc

## Resource Recommendation (Agentic RAG)

In [74]:
rag_input = f"""
User Situation:
{example_situation}
Current SMART goals: 
{results}
Guidance for Relevant Resources:
{resource_reasoning}
"""
retrieved_resources = analyze_situation_rag(rag_input, k=15)

In [75]:
retrieved_resources

"our Lady of Mount Carmel, URL: https://www.olmcridgewood.com/, Phone: 2014442000, Description: Our Lady of Mount Carmel is a vibrant parish community offering a wide range of spiritual, educational, and social services. Key offerings include worship services with a diverse Mass schedule, adult and youth faith formation, and various ministries like music and outreach. Special programs such as the Difficult Conversations Series and CYO sports foster community engagement. The parish also hosts the Academy of Our Lady, providing education for children. Perpetual Eucharistic Adoration allows for continuous prayer, promoting mental and spiritual well-being. Additionally, resources for sacramental preparations and support for families are readily available, ensuring comprehensive care for all parishioners.\nNJ Reentry Corporation, URL: https://njreentry.org/, Phone: nan, Description: The New Jersey Reentry Corporation (NJRC) is a non-profit organization dedicated to helping individuals reint

In [76]:
def resource_selection_prompt(situation, meta_reasoning, retrieved_resources):
    return f"""
You are an expert resource recommendation assistant. Given the user's specific situation, conceptual guidance, and a list of potential resources retrieved from a knowledge base, your task is to carefully select and recommend ONLY the most relevant and helpful resources. 

Consider the original situation and the conceptual guidance carefully to ensure selected resources specifically match the user's needs and context.

Do not make anything up. Only select resources from the list that is given to you.

## Original User Situation:
{situation}

## Conceptual Guidance for Selecting Resources:
{meta_reasoning}

## Retrieved Resources:
{retrieved_resources}

## Your task:
- Carefully evaluate the retrieved resources in relation to the user's situation and the provided conceptual guidance.
- Select and recommend ONLY the resources that BEST match the user's situation.
- Clearly present your recommendations in a structured, easy-to-read format.
- For each selected resource, briefly explain why it is specifically relevant.

## Selected Most Relevant Resources (clearly listed and explained):
"""

def select_relevant_resources(situation, meta_reasoning_guidance, retrieved_resources):
    prompt = resource_selection_prompt(situation, meta_reasoning_guidance, retrieved_resources)
    messages = [
        {"role": "system", "content": "You are an expert assistant selecting highly relevant resources tailored to the user's situation."},
        {"role": "user", "content": prompt}
    ]

    selected_resources_response = call_llm(messages)
    
    return selected_resources_response

In [85]:
def format_selected_resources(raw_text):
    formatted_text = raw_text.replace("\\n", "\n")
    formatted_text = re.sub(r'\*\*', '', formatted_text)
    formatted_text = re.sub(r'###+', '', formatted_text)
    formatted_text = re.sub(r'\s*(\d+)\.\s*(.+?):\s*\n', r'\n\1. **\2**\n', formatted_text)
    formatted_text = re.sub(r'–\s*URL\s*:', '- **URL:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Phone\s*:', '- **Phone:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Relevance\s*:', '- **Relevance:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'\n+', '\n', formatted_text).strip()
    return formatted_text

In [83]:
selected_resources = select_relevant_resources(
        example_situation,
        resource_reasoning,
        retrieved_resources)
resource_output = format_selected_resources(selected_resources)
print(resource_output)

## Selected Most Relevant Resources
1. Complete Care NJ  
   URL: [Complete Care NJ](https://completecarenj.org/)  
   Phone: 6094650258  
   Relevance: Complete Care NJ is a Federally Qualified Health Center that offers comprehensive healthcare services, including primary medical care, mental health services, and specialized care for seniors. Their focus on accessibility regardless of income or insurance status makes it a suitable option for someone dually enrolled in Medicaid/Medicare. This resource could support the user's physical wellness needs through dietary advice and health management, while also facilitating social engagement opportunities through community programs.
2. North Jersey Friendship House  
   URL: [North Jersey Friendship House](https://njfriendshiphouse.org/)  
   Phone: 2014882121  
   Relevance: The North Jersey Friendship House provides personalized services and programs aimed at social, emotional, and vocational goals. Given the user's struggle with isolation

# Follow-up Questions

In [86]:
def followup_questions_prompt(situation, smart_goals, selected_resources, meta_reasoning_followup):
    return f"""
You are an expert mental health support assistant tasked with formulating insightful follow-up questions. Your objective is to ask clear, targeted, and supportive questions that will help clarify the user's situation and guide next steps.

Consider all of the following contextual information carefully to formulate your follow-up questions:

### User's Initial Situation:
{situation}

### SMART Goals Suggested for the User:
{smart_goals}

### Selected Relevant Resources for the User:
{selected_resources}

### Conceptual Guidance for Follow-up Questions:
{meta_reasoning_followup}

---

### Your Task:
- Formulate clear, insightful follow-up questions that address important gaps, seek clarity, and help in guiding the user's next steps effectively.
- Questions should be empathetic, supportive, and directly relevant to the provided context.

### Insightful Follow-up Questions:
"""

In [87]:
def generate_followup_questions(situation, smart_goals, selected_resources, meta_reasoning_followup):
    prompt = followup_questions_prompt(
        situation, smart_goals, selected_resources, meta_reasoning_followup
    )

    messages = [
        {"role": "system", "content": "You are an insightful assistant skilled in asking empathetic, supportive, and targeted follow-up questions for mental health support."},
        {"role": "user", "content": prompt}
    ]

    followup_questions_response = call_llm(messages)

    return followup_questions_response

In [90]:
followup_questions = generate_followup_questions(
        example_situation,
        results,
        resource_output,
        question_reasoning
    )

print(followup_questions)

1. **Exploring Current Routines**: Can you share what your current daily routines look like regarding diet and exercise? What challenges do you face in initiating changes in those areas?

2. **Understanding Motivation**: What motivates you the most to want to improve your physical wellness and build social connections? How do you feel these changes might impact your daily life?

3. **Identifying Barriers**: What do you think is preventing you from being more active or trying new social activities? Are there specific fears or concerns that you have?

4. **Community Engagement**: Have you participated in any local community activities or events before? If so, what did you enjoy, or what stopped you from continuing?

5. **Social Connection Preferences**: What types of social activities do you think would interest you the most? Would you prefer structured activities like classes, or more casual interactions like group walks or coffee meetups?

6. **Physical Limitations**: Are there any phy

# Step 3: Aggregate Responses and Draft Holistic Output

In [94]:
def holistic_response_prompt(situation, smart_goals, resources, followup_questions):
    return f"""
You are an empathetic and supportive mental health assistant. Your task is to integrate the provided SMART goals, recommended resources, and insightful follow-up questions into a compassionate, cohesive, and clear holistic response to the user's situation. Begin your response with empathy, validating the user's feelings and concerns.

### User's Situation:
{situation}

### Suggested SMART Goals:
{smart_goals}

### Recommended Resources:
{resources}

### Insightful Follow-up Questions:
{followup_questions}

---

### Your Task:
Craft a holistic response:
- Start with empathetic validation and encouragement.
- Clearly and supportively summarize recommended SMART goals.
- Highlight the relevant resources clearly, explaining briefly how they can help. Make sure to include contact information that has been given to you. 
- Present insightful follow-up questions, encouraging the user to engage further.
- Maintain a supportive and warm tone throughout.

### Final Holistic Response:
"""

def generate_holistic_response(situation, smart_goals, resources, followup_questions):
    prompt = holistic_response_prompt(
        situation, smart_goals, resources, followup_questions
    )

    messages = [
        {"role": "system", "content": "You are a compassionate mental health support assistant, providing empathetic, structured, and actionable guidance."},
        {"role": "user", "content": prompt}
    ]

    holistic_response = call_llm(messages)

    return holistic_response


In [95]:
final_response = generate_holistic_response(
        example_situation,
        results,
        resource_output,
        followup_questions
    )

print("\n=== Final Holistic Response ===\n")
print(final_response)


=== Final Holistic Response ===

I want to begin by acknowledging how challenging it can feel to navigate feelings of isolation, especially at this stage in life. It's admirable that you're seeking to focus on your physical wellness and build social connections; those are significant steps toward enhancing your well-being. Your willingness to take action speaks volumes about your strength and resilience.

To help you on this journey, I’ve outlined some SMART goals tailored specifically for your aspirations:

**1. Physical Exercise Goal:**  
Aim to engage in 30 minutes of moderate physical activity every day, such as walking around your neighborhood or utilizing local parks in Morristown. Start at a comfortable pace and gradually increase your activity level. Track your walks in a simple journal or a step counter app. Commit to this for the next 4 weeks, and then reassess.

**2. Nutrition and Diet Improvement Goal:**  
Incorporate at least 2 servings of fresh fruits or vegetables into 

# Step 4: Responses Update

In [97]:
def interpret_user_request(user_input):
    prompt = f"""
Given the user's input below, determine clearly and explicitly which part(s) of the holistic response needs updating. Your response should only select from the following categories:
- SMART Goals
- Resources
- Follow-up Questions

User Input:
{user_input}

Respond ONLY with the categories that clearly need updating (comma-separated if multiple). If unclear, default to 'Follow-up Questions'.
"""
    messages = [
        {"role": "system", "content": "You are an analytical assistant who identifies clearly which parts of the holistic response the user wants updated."},
        {"role": "user", "content": prompt}
    ]

    response = call_llm(messages).lower()

    # Parse response into a clean list
    modules_to_update = []
    for module in ['smart goals', 'resources', 'follow-up questions']:
        if module in response:
            modules_to_update.append(module)

    if not modules_to_update:
        modules_to_update = ['follow-up questions']

    return modules_to_update


In [138]:
def extract_section(response, section_name):
    pattern = rf"{section_name}:(.*?)(\n\n|$)"
    match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
    return match.group(1).strip() if match else ""
    
def update_holistic_response(current_response, user_input, initial_situation, meta_reasoning_result):
    modules_needed = interpret_user_request(user_input)
    print(modules_needed)

    updated_smart_goals = None
    updated_resources = None
    updated_followup_questions = None

    # Enhance the situation by explicitly incorporating user feedback
    enhanced_situation = f"{initial_situation}\n\nAdditional user feedback/request: {user_input}"

    # SMART Goals update (if needed)
    if 'smart goals' in modules_needed:
        smart_goal_reasoning = meta_reasoning_result["SMART Goals Module"] + f"\n\nUser Feedback: {user_input}"
        updated_smart_goals = get_smart_goals(smart_goal_reasoning, enhanced_situation)

    # Resources update (if needed)
    if 'resources' in modules_needed:
        resource_reasoning = meta_reasoning_result["Resources Module"] + f"\n\nUser Feedback: {user_input}"
    
        # Incorporate current SMART goals for richer context
        current_smart_goals = updated_smart_goals or extract_section(current_response, "SMART Goals")
    
        rag_input = f"""User Situation:{initial_situation}\n\n Guidance for Relevant Resources:{resource_reasoning}\n\n\
                       Current SMART Goals:{current_smart_goals}\
                      \n\nAdditional User Feedback:{user_input}"""
        
        retrieved_resources = analyze_situation_rag(rag_input)
        print(retrieved_resources)
        
        updated_resources = select_relevant_resources(enhanced_situation, resource_reasoning, retrieved_resources)


    # Follow-up Questions update (if needed)
    if 'follow-up questions' in modules_needed:
        followup_reasoning = meta_reasoning_result["Follow-up Questions Module"] + f"\n\nUser Feedback: {user_input}"

        # Use updated SMART goals and resources if available, else extract from current_response
        smart_goals_for_followup = updated_smart_goals or extract_section(current_response, "SMART Goals")
        resources_for_followup = updated_resources or extract_section(current_response, "Recommended Resources")

        updated_followup_questions = generate_followup_questions(
            enhanced_situation,
            smart_goals_for_followup,
            resources_for_followup,
            followup_reasoning
        )
        
    if any([updated_smart_goals, updated_resources, updated_followup_questions]):
        summary_parts = []
        if updated_smart_goals:
            summary_parts.append(f"### Updated SMART Goals:\n{updated_smart_goals}")
        if updated_resources:
            summary_parts.append(f"### Updated Resources:\n{updated_resources}")
        if updated_followup_questions:
            summary_parts.append(f"### Updated Follow-up Questions:\n{updated_followup_questions}")
    
        formatted_update_prompt = f"""You are a helpful assistant. Given the following updated content for the user, rewrite it as a supportive and clearly formatted update message. Present each section in a warm, readable tone that integrates smoothly with the previous holistic response. Do not repeat the entire holistic response — only show the updated content in a well-structured and polished way.You may use bullets and markdown formatting, but avoid using emojis in headers.
                                {'\n\n'.join(summary_parts)}
                                ---
                                Return the updated response in a clean and user-friendly format.
                                """
        messages = [
            {"role": "system", "content": "You are an empathetic mental health support assistant."},
            {"role": "user", "content": formatted_update_prompt}
        ]
        
        formatted_updates = call_llm(messages)
        return formatted_updates
    else:
        return "There were no updates based on the user's request."

In [140]:
current_holistic_response = final_response
user_input = "give me more smart goals that I can follow to improve my physicall wellness"
initial_situation = example_situation

new_response = update_holistic_response(
        current_holistic_response,
        user_input,
        initial_situation,
        meta_reasoning_result)
print("\n=== Updated Holistic Response ===\n")
print(new_response)

['smart goals']

=== Updated Holistic Response ===

### Updated SMART Goals

Here are some revisited SMART goals aimed at enhancing physical wellness through improved diet and exercise, specifically tailored for a 72-year-old retired man:

#### Goal 1: Increase Physical Activity
- **Specific**: Walk for at least 20 minutes each day, five days a week.
- **Measurable**: Use a notebook or smartphone app to track the duration and frequency of walks.
- **Achievable**: Begin with shorter walks (10-15 minutes) if necessary, gradually increasing to 20 minutes.
- **Realistic**: Choose safe and accessible walking locations, such as local parks or community centers.
- **Timely**: Reach the 20 minutes per day target within 4 weeks, with weekly progress assessments.

#### Goal 2: Improve Nutritional Intake
- **Specific**: Include at least one serving of fruits and/or vegetables in each meal.
- **Measurable**: Maintain a meal log to document daily fruit and vegetable consumption.
- **Achievable**: S

In [122]:
# new_sit = "A trans man in Jersey City, NJ, aged 35, \
# recently moved into permanent housing after recovering \
# from substance use and trauma related to domestic violence. \
# He is employed part-time, has ID, and seeks support with maintaining mental \
# health and building community connections. He also wants to pursue a restraining order against his former partner."

# rag_input = f"""
# User Situation:
# {new_sit}
# Current SMART goals: 
# {results}
# Guidance for Relevant Resources:
# {resource_reasoning}
# """
# retrieved_resources = analyze_situation_rag(rag_input, k=15)

In [124]:
# selected_resources = select_relevant_resources(
#         new_sit,
#         resource_reasoning,
#         retrieved_resources)
# resource_output = format_selected_resources(selected_resources)
# print(resource_output)

## Selected Most Relevant Resources
1. North Jersey Recovery Center  
   - URL: [North Jersey Recovery](https://northjerseyrecovery.com/)  
   - Phone: 877-632-5541  
   - Relevance: This resource is crucial for the user as it specializes in comprehensive services for substance use recovery. Given the user's background of recovering from substance use, the North Jersey Recovery Center offers tailored outpatient programs and counseling that can provide ongoing support as he navigates his mental health and recovery journey. The holistic approach could be beneficial in addressing any related trauma from past experiences.
2. NJ Reentry Corporation  
   - URL: [NJ Reentry Corporation](https://njreentry.org/)  
   - Relevance: This organization is dedicated to reintegrating individuals who have faced issues with addiction treatment and can offer resources that include mental health services, legal assistance, and community support. Given the user's interest in pursuing a restraining order, t